### Task 2  SKETCHING TASK 

TO DO
1. Build code for computing a Count-Min sketch,
2. play with different heights and widths for the Count-Min sketch matrix. 
3. Compare it to the MIN_WISE sampling strategy. 
4. Is it more space-efficient/accurate? What about run-time? 
5. Use the theory to explain any differences you observe.

ALGORITHM: COUNT - MIN SKETCH

count_min_sketch (over_count, return_minimum value, support subtraction)
    1. Initialize the 2D d x w array of counters
    2. Pick "d" hash functions for the i-th item
    3. For every hash in the hash function 
        counter of h(item) = counter of h(item) + 1
    4. For any "item" 
        report the minimum value of the h(item) as the estimation of frequency
    
    
item is ip_address
h is hash function 

In [40]:
#import necessary libraries
import pandas as pd
import mmh3
import murmurhash
from bitarray import bitarray
import numpy as np
import math

In [41]:
#Read the dataset of CTU-13 malware 43
dataset = pd.read_csv('dataset/capture20110811.pcap.netflow.labeled', delim_whitespace=True, chunksize=1000)
first_chunk = dataset.get_chunk()

# preprocess: clean up columns
def preprocess(chunk):
    chunk = chunk.rename(index=str, columns={
        "#Date_flow": "Date",
        "start": "Start",
        "Durat": "Duration",
        "Prot":"Protocol",
        "Src_IP_Addr:Port" : "Source_IP",
        "Dst_IP_Addr:Port": "Destination_IP",
        "Label(LEGITIMATE:Botnet:Background)": "Label"})
    chunk = chunk.drop(axis="columns", labels="->")
    
    chunk['Source_IP'] = chunk['Source_IP'].apply(lambda x: x.split(":")[0])
    chunk['Destination_IP'] = chunk['Destination_IP'].apply(lambda x: x.split(":")[0])
    return chunk

first_chunk = preprocess(first_chunk)
first_chunk.head()

,Date,Start,Duration,Protocol,Source_IP,Destination_IP,Flags,Tos,Packets,Bytes,Flows,Label
0,2011-08-11,10:10:00.003,0.967,UDP,89.31.8.11,147.32.84.229,INT,0,2,135,1,Background
1,2011-08-11,10:10:00.003,0.967,UDP,147.32.84.229,89.31.8.11,INT,0,2,276,1,Background
2,2011-08-11,10:10:00.006,0.000,UDP,208.88.186.6,147.32.84.229,INT,0,1,62,1,Background
3,2011-08-11,10:10:00.008,0.000,UDP,92.118.218.77,147.32.84.229,INT,0,1,78,1,Background
4,2011-08-11,10:10:00.009,0.000,UDP,182.185.139.181,147.32.84.229,INT,0,1,72,1,Background


In [42]:
len(first_chunk['Source_IP'].unique()) #406

406

In [43]:
len(first_chunk['Source_IP'])#1000
#first_chunk['Source_IP'] = '147.32.84.229';

1000

In [ ]:
#1. Initialize the 2D d x w array of counters
# TO-DO: play around with this d and w (d=3 W=4; d=5 W=100;d=5 W=1000;d=5- W=100)
d = 50; #rows - hash
W = 100; #columns - ip
CMS = [0] * d
for i in range(d):
    CMS[i] = [0] * W
#print(CMS)
# create a hash function to fill CMS with values
# hash
# CMS[0][1] =1;
# print(CMS);

In [ ]:
# read through the stream
# this code will take some time to read all the data, you can stop it in the middle and proceed to next code
chunks = pd.read_csv('dataset/capture20110811.pcap.netflow.labeled', delim_whitespace=True, chunksize=1000)
chunk_count = 0
for chunk in chunks:
    print("Processing chunk ",chunk_count)
    chunk = preprocess(chunk)
    for index, item in chunk.iterrows():
        ip_address = item["Source_IP"]
        # apply d hash function to the IP address
        for i in range(d):
            j = mmh3.hash(ip_address, i) % W # we use modulo here so the resulting hash is less than W
            # update the CMS matrix
            CMS[i][j] = CMS[i][j] + 1
        
        # do the same for destination IP
        ip_address = item["Destination_IP"]
        for i in range(d):
            j = mmh3.hash(ip_address, i) % W # we use modulo here so the resulting hash is less than W
            # update the CMS matrix
            CMS[i][j] = CMS[i][j] + 1
    chunk_count = chunk_count + 1

Processing chunk  0
Processing chunk  1
Processing chunk  2
Processing chunk  3
Processing chunk  4
Processing chunk  5
Processing chunk  6
Processing chunk  7
Processing chunk  8
Processing chunk  9
Processing chunk  10
Processing chunk  11
Processing chunk  12
Processing chunk  13
Processing chunk  14
Processing chunk  15
Processing chunk  16
Processing chunk  17
Processing chunk  18
Processing chunk  19
Processing chunk  20
Processing chunk  21
Processing chunk  22
Processing chunk  23
Processing chunk  24
Processing chunk  25
Processing chunk  26
Processing chunk  27
Processing chunk  28


In [ ]:
# read frequency of certain IP address

# see the min-wise sampling task result of top 10 frequent IP
ip_list = ["147.32.84.229", "147.32.80.9", "147.32.84.138", "147.32.84.59", "147.32.86.165", "147.32.80.13", "147.32.85.25", "147.32.84.118", "147.32.84.165", "147.32.84.171", "147.32.85.124"]

for ip in ip_list:
    count = None
    for i in range(d):
        j = mmh3.hash(ip, i) % W
        if count is None:
            count = CMS[i][j]
        else:
            count = min(count, CMS[i][j]) # we use the minimum of all value in the corresponding hash function

    print(ip, "\t", count)